# Simple Linear Program

Let's solve the following linear program:

\begin{array}{ll}
  \min       & 2 x_1 + 3 x_2\\
  \mathrm{s.t.} & 3 x_1 + 4 x_2 \geq 1\\
             & x_1, x_2 \geq 0
\end{array}

Before we start, what are the model "parameters"? Let's solve in Excel so we can verify our Pyomo model.

### Pyomo model, one chunk at a time

#### Note:  If you try to re-run certain cells you might get an error.  If so, start evaluating cells from the beginning (here).

Import the `pyomo.environ` model as `pe`. 

In [19]:
import pyomo.environ as pe

Create a new (and empty) pyomo model.  The "concrete" means we will hard-code parameter values.

In [20]:
model = pe.ConcreteModel()
print(type(model))

<class 'pyomo.core.base.PyomoModel.ConcreteModel'>


First let's define the decision variables.  We'll give the decisions specific indexes and specify that they be nonnegative.

In [21]:
# We have to tell pyomo how many dv's we have using indexes.  This is a step you will have to
# do yourself most times!  Indexes can be numbers or strings.
x_indexes = [1, 2]

In [22]:
# Create a variable using the indexes and specifying the type of variable, here nonnegative real numbers, x >= 0.
# The model.x says to attach the variable to the model and call it "x".
model.x = pe.Var(x_indexes, domain=pe.NonNegativeReals)
print(type(model.x))

<class 'pyomo.core.base.var.IndexedVar'>


In [23]:
# Define the objective function value as z = 2x1 + 3x2 and attach it to the model as an attribute obj.
# By default, the objective is set to be a "min" problem.
model.obj = pe.Objective(expr = 2*model.x[1] + 3*model.x[2])
print(type(model.obj))

<class 'pyomo.core.base.objective.SimpleObjective'>


In [24]:
# Create a constraint "3x1 + 4x2 >= 1" and store attach it to the model as Constraint1.
model.constraint = pe.Constraint(expr = 3*model.x[1] + 4*model.x[2] >= 1)
print(type(model.constraint))

<class 'pyomo.core.base.constraint.SimpleConstraint'>


In [25]:
# Instantiate the model instance and run the solver.  This will basically always be the same
# for all models that you run!  You can set tee=False to suppress the output.

opt = pe.SolverFactory('glpk')
success = opt.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/j6/vzm_9p_j4_x08rrgkr0f7cb84vgyj9/T/tmpvqrtydru.glpk.raw
 --wglp /var/folders/j6/vzm_9p_j4_x08rrgkr0f7cb84vgyj9/T/tmpf3gt599r.glpk.glp
 --cpxlp /var/folders/j6/vzm_9p_j4_x08rrgkr0f7cb84vgyj9/T/tmplno37ara.pyomo.lp
Reading problem data from '/var/folders/j6/vzm_9p_j4_x08rrgkr0f7cb84vgyj9/T/tmplno37ara.pyomo.lp'...
2 rows, 3 columns, 3 non-zeros
21 lines were read
Writing problem data to '/var/folders/j6/vzm_9p_j4_x08rrgkr0f7cb84vgyj9/T/tmpf3gt599r.glpk.glp'...
15 lines were written
GLPK Simplex Optimizer, v4.65
2 rows, 3 columns, 3 non-zeros
Preprocessing...
1 row, 2 columns, 2 non-zeros
Scaling...
 A: min|aij| =  3.000e+00  max|aij| =  4.000e+00  ratio =  1.333e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
      0: obj =   0.000000000e+00 inf =   1.000e+00 (1)
      1: obj =   7.500000000e-01 inf =   0.000e+00 (0)
*     2: obj =   6.66

In [26]:
model.display()

Model unknown

  Variables:
    x : Size=2, Index=x_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          1 :     0 : 0.333333333333333 :  None : False : False : NonNegativeReals
          2 :     0 :               0.0 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 0.666666666666666

  Constraints:
    constraint : Size=1
        Key  : Lower : Body               : Upper
        None :   1.0 : 0.9999999999999989 :  None


In [27]:
# Get the objective function value from the optimal solution and print it.
obj_val = model.obj.expr()
print('Optimal objective value = {}'.format(obj_val))

Optimal objective value = 0.666666666666666


In [28]:
# Get a list of optimal decision variable values.
x = []  # create an empty list to store values
for index in model.x_index.value_list:
    x.append(model.x[index].value)
print('optimal x = {}'.format(x))

optimal x = [0.333333333333333, 0.0]


In [30]:
# Let's print the constraint
model.constraint.pprint()       # prints the mathematical constraint
model.constraint.display()      # prints the value of the Lower and Body (think, RHS and LHS)

constraint : Size=1, Index=None, Active=True
    Key  : Lower : Body            : Upper : Active
    None :   1.0 : 3*x[1] + 4*x[2] :  +Inf :   True
constraint : Size=1
    Key  : Lower : Body               : Upper
    None :   1.0 : 0.9999999999999989 :  None


In [32]:
# Get the slack in the constraint
print('constraint slack = {}'.format(model.constraint.slack()))

constraint slack = -1.1102230246251565e-15
